In [18]:
from pathlib import Path
from tqdm.notebook import tqdm
import yaml
from collections import Counter
import pandas as pd
from omnibelt import load_json, save_json, load_csv, load_yaml, load_csv_rows
from omnifin import misc

In [6]:
root = misc.repo_root() / 'extra' / 'usbank'

In [7]:
locpath = root / 'cashplus_locs.csv'
recpath = root / 'cashplus_records.csv'
locpath.exists(), recpath.exists()

(True, True)

In [22]:
locs = list(load_csv_rows(locpath))
len(locs)

54

In [43]:
# locs[0]

In [21]:
recs = list(load_csv_rows(recpath))
len(recs)

54

In [44]:
# recs[0]

In [39]:
def convert_entry(loc, rec):
	if rec['Amount'] > 0:
		return None
	
	item = {
		'usd': -rec['Amount'],
		'txn-number': rec['Memo'].split(';')[0].strip(),
		'mcc': rec['Memo'].split(';')[1].strip(),
		'cleaned': loc['Details'],
	}
	desc = rec['Name']
	if 'EURO' in desc:
		amount = desc.split(' - ')[-1].split('EURO')[0].strip()
		amount = float(amount.replace(',', ''))
		item.update({
			'received-amount': amount,
			'received-unit': 'EURO',
		})
	
	city = loc['City']
	if str(city) == 'nan':
		city = None
	location = loc['Country']
	if str(location) == 'nan':
		location = None
	
	item.update({
		'date': rec['Date'],
		'city': city,
		'location': location,
		'online': loc['Type'] == 'online',
	})
	return item

fixed = []
for rec, loc in zip(recs, locs):
	item = convert_entry(loc, rec)
	if item is None:
		continue
	fixed.append(item)
len(fixed)

41

In [42]:
fixedpath = root / 'fixed.json'
save_json(fixed, fixedpath)

54